In [103]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import plotly.io as pio
from wordcloud import *
from functools import reduce
from PIL import Image
pd.set_option('display.max_rows', 200)
from statsmodels.graphics.mosaicplot import mosaic
from plotly.subplots import make_subplots
pio.templates.default = "simple_white"

path = './'

#complete DFs
rhz_artworks = pd.read_pickle(path+'Rhizome_data/rhizome_artworks_extra.pkl')
rhz_artists = pd.read_pickle(path+'Rhizome_data/rhizome_artists_extra.pkl')
moma_artists = pd.read_pickle(path+'MOMA_data/pickle/MoMAArtists.pkl')
moma_artworks = pd.read_pickle(path+'MOMA_data/pickle/MoMAartworks.pkl')
moma_artworks_old =  pd.read_pickle(path+'MOMA_data/pickle/old_artworks.pkl')
moma_artworks_old.loc[:,'DateAcquired'] = moma_artworks_old['DateAcquired'].where((moma_artworks_old['DateAcquired'].str.len() <= 4), moma_artworks_old['DateAcquired'].str[0:4])
moma_artworks_old['DateAcquired'] = moma_artworks_old['DateAcquired'].replace('nan', str('0'))
moma_artworks_old['DateAcquired']=moma_artworks_old['DateAcquired'].astype('int')
moma_artworks_new = pd.read_pickle(path+'MOMA_data/pickle/new_artworks.pkl')
moma_artworks_new.loc[:,'DateAcquired'] = moma_artworks_new['DateAcquired'].where((moma_artworks_new['DateAcquired'].str.len() <= 4), moma_artworks_new['DateAcquired'].str[0:4])
moma_artworks_new['DateAcquired'] = moma_artworks_new['DateAcquired'].replace('nan', str('0'))
moma_artworks_new['DateAcquired']=moma_artworks_new['DateAcquired'].astype('int')
moma_artworks = pd.concat([moma_artworks_old, moma_artworks_new])
moma_artworks_old = moma_artworks.loc[moma_artworks['DateCreated'] <= 1982]
moma_artworks_new = moma_artworks.loc[moma_artworks['DateCreated'] >= 1983]
moma_rhz_compare = moma_artworks_new.loc[moma_artworks_new['DateAcquired'] >= 2000]
moma_rhz_compare = moma_rhz_compare.loc[moma_rhz_compare['DateCreated'] >= 1983]

#MoMA department DFs
moma_draw = moma_artworks.loc[moma_artworks['Department'] == 'Drawings & Prints']
moma_draw_before = moma_draw.loc[moma_draw['DateCreated'] <= 1982]
moma_draw_after = moma_draw.loc[moma_draw['DateCreated'] >= 1983]
moma_films = moma_artworks.loc[moma_artworks['Department'] == 'Film']
moma_films_before = moma_films.loc[moma_films['DateCreated'] <= 1982]
moma_films_after = moma_films.loc[moma_films['DateCreated'] >= 1983]
moma_media = moma_artworks.loc[moma_artworks['Department'] == 'Media and Performance']
moma_media_before = moma_media.loc[moma_media['DateCreated'] <= 1982]
moma_media_after = moma_media.loc[moma_media['DateCreated'] >= 1983]
moma_photo = moma_artworks.loc[moma_artworks['Department'] == 'Photography']
moma_photo_before = moma_photo.loc[moma_photo['DateCreated'] <= 1982]
moma_photo_after = moma_photo.loc[moma_photo['DateCreated'] >= 1983]

#Rhizome with text
rhizome_txt_clean = pd.read_pickle(path+'Rhizome_data/rhizome_artworks_extra_text_clean.pkl')
rhizome_txt_stop_kw = pd.read_pickle(path+'Rhizome_data/rhizome_artworks_extra_text_clean_stop_keywords.pkl')

#MoMA with text
moma_arch_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_cont_text_final.pkl')
moma_arch_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_mod_text_only_final.pkl')
moma_draw_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/draws_prints_cont_text_final.pkl')
moma_draw_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/draws_prints_mod_text_final.pkl')
moma_films_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/films_cont_text_final.pkl')
moma_films_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/films_mod_text_final.pkl')
moma_fluxus_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/fluxus_cont_text_final.pkl')
moma_fluxus_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/fluxus_mod_text_final.pkl')
moma_media_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/media_perf_cont_text_final.pkl')
moma_media_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/media_perf_mod_text_final.pkl')
moma_paint_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/paint_sculp_cont_text_final.pkl')
moma_paint_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/paint_sculp_mod_text_final.pkl')
moma_photo_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/photo_cont_text_final.pkl')
moma_photo_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/photo_mod_text_final.pkl')

#moma text stop and keywords
moma_arch_cont_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_cont_text_final_stop.pkl')
moma_arch_mod_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_mod_text_only_final_stop.pkl')
moma_draw_cont_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/draws_prints_cont_text_final_stop.pkl')
moma_draw_mod_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/draws_prints_mod_text_final_stop.pkl')
moma_films_cont_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/films_cont_text_final_stop.pkl')
moma_films_mod_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/films_mod_text_final_stop.pkl')
moma_media_cont_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/media_perf_cont_text_final_stop.pkl')
moma_media_mod_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/media_perf_mod_text_final_stop.pkl')
moma_paint_cont_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/paint_sculp_cont_text_final_stop.pkl')
moma_paint_mod_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/paint_sculp_mod_text_final_stop.pkl')
moma_photo_cont_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/photo_cont_text_final_stop.pkl')
moma_photo_mod_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/photo_mod_text_final_stop.pkl')
moma_full_text_stop = pd.concat([moma_arch_cont_text_stop, moma_draw_cont_text_stop, moma_draw_mod_text_stop, moma_films_cont_text_stop, moma_films_mod_text_stop, moma_media_cont_text_stop, moma_media_mod_text_stop, moma_paint_cont_text_stop, moma_paint_mod_text_stop, moma_photo_cont_text_stop, moma_photo_mod_text_stop])

In [180]:
moma_media_after

,Title,Artist,ID,DateCreated,Medium,Department,DateAcquired,URL,ThumbnailURL,Nationality,Gender
75280,Inasmuch As It Is Always Already Taking Place,Gary Hill,7416,1990,"Sixteen-channel video (black and white, sound)...",Media and Performance,1997,http://www.moma.org/collection/works/81321,http://www.moma.org/media/W1siZiIsIjcyOTQ1Il0s...,American,M
75589,Stations,Bill Viola,7898,1994,"Five-channel video (color, sound), five granit...",Media and Performance,1997,http://www.moma.org/collection/works/81737,http://www.moma.org/media/W1siZiIsIjEwNjY4MiJd...,American,M
78994,My New Theater 1,Joan Jonas,2930,1997,"Video (color, sound) and mixed media in miniature",Media and Performance,2001,http://www.moma.org/collection/works/85900,http://www.moma.org/media/W1siZiIsIjI4OTE2OSJd...,American,F
79844,The Forty Part Motet (A reworking of “Spem in ...,Janet Cardiff,8473,2001,"Forty-track sound recording, forty speakers",Media and Performance,2002,http://www.moma.org/collection/works/87291,http://www.moma.org/media/W1siZiIsIjk4OTI0Il0s...,Canadian,F
80919,What You Mean We?,Laurie Anderson,6807,1986,"Video (color, sound)",Media and Performance,1986,http://www.moma.org/collection/works/88831,http://www.moma.org/media/W1siZiIsIjQwMDUxMCJd...,American,F
...,...,...,...,...,...,...,...,...,...,...,...
136206,Sweet Desire a.k.a. Burial Piece (window version),Pope.L,37145,1996,"Video (Color, sound); 2:17 minutes",Media and Performance,2019,http://www.moma.org/collection/works/401210,http://www.moma.org/media/W1siZiIsIjQ3MTc1NSJd...,American,M
136224,How to Make Money Religiously,Laure Prouvost,131903,2014,"Video (color, sound)",Media and Performance,2019,http://www.moma.org/collection/works/401355,missing,French,F
136264,VVEBCAM,Petra Cortright,131907,2007,"Webcam video (color, sound)",Media and Performance,2019,http://www.moma.org/collection/works/401436,http://www.moma.org/media/W1siZiIsIjQ3MjE1OSJd...,American,F
136689,Projects 195: Park McArthur,Park McArthur,48556,2018,missing,Media and Performance,2019,missing,missing,American,F


In [4]:
def group_genders(df, collection):
    if collection == 'MoMA':
        collectives = ['M', 'F', 'missing', 'NB']
    else:
        collectives = ['M', 'F', 'missing', 'N/B']
    df = df['Gender'].value_counts().rename_axis('Gender').reset_index(name='Counts')
    df_collectives = ['collectives', df[~df.Gender.isin(collectives)]['Counts'].sum()]
    df = df.loc[~df.Gender.isin(collectives) == False].reset_index(drop=True)
    df.loc[5] = df_collectives
    df = df.sort_values(by='Counts', ascending=False).replace('M', 'Male').replace('F', 'Female')
    return df

def genders_over_time(df, time_var, time_title, collection):
    if collection == 'MoMA':
        nonbinary = 'NB'
    else: 
        nonbinary = 'N/B'
    df_m = df.loc[df['Gender'] == 'M']
    df_m = df_m[time_var].value_counts().rename_axis(time_title).reset_index(name='Male Artists')
    df_f = df.loc[df['Gender'] == 'F']
    df_f = df_f[time_var].value_counts().rename_axis(time_title).reset_index(name='Female Artists')
    df_merge = pd.merge(df_m, df_f, on=time_title, how='outer')
    df_nb = df.loc[df['Gender'] == nonbinary]
    df_nb = df_nb[time_var].value_counts().rename_axis(time_title).reset_index(name='Non Binary Artists')
    df_col = df[(df['Gender'] != 'M') & (df['Gender'] != 'F') & (df['Gender'] != 'missing') & (df['Gender'] != nonbinary)]
    df_col = df_col[time_var].value_counts().rename_axis(time_title).reset_index(name='Collectives')
    df_missing = df.loc[df['Gender'] == 'missing']
    df_missing = df_missing[time_var].value_counts().rename_axis(time_title).reset_index(name='Artists with Missing Gender')
    df_extra = pd.merge(df_nb, df_col, on=time_title, how='outer')
    df_merge = pd.merge(df_merge, df_extra, on=time_title, how='outer')
    df_merge = pd.merge(df_merge, df_missing, on=time_title, how='outer')
    df_merge = df_merge.fillna('0')
    df_merge = df_merge.loc[df_merge[time_title] != 0]
    df_merge = df_merge.astype(int)
    df_merge = df_merge.sort_values(by=time_title).reset_index(drop=True)
    return df_merge


**GENDER BREAKDOWN PIE CHARTS RHZ VS MoMA FULL + RHZ VS MoMA SAMPLED**

In [178]:
#create a DF w/ genders and counts, group collectives together
rhz_gender_rep = group_genders(rhz_artworks, 'Rhizome')
moma_gender_rep = group_genders(moma_artworks, 'MoMA')

fig_1 = px.pie(rhz_gender_rep, values='Counts', names='Gender',
             title='Gender Breakdown of Rhizome Collection')
fig_1.update_traces(textposition='auto', textinfo='label+percent', hoverinfo='label+value+percent', hovertemplate=None, pull=[0, 0, 0, 0.2])
fig_1.update_layout(width=800)

fig_2 = px.pie(moma_gender_rep, values='Counts', names='Gender',
             title='Gender Breakdown of MoMA Collection')
fig_2.update_traces(textposition='auto', textinfo='label+percent', hoverinfo='label+value+percent', hovertemplate=None, pull=[0, 0, 0, 0.2])
fig_2.update_layout(width=800)

fig_1.show(), fig_2.show()

(None, None)

In [179]:
#create a DF w/ genders and counts, group collectives together
rhz_gender_rep = group_genders(rhz_artworks, 'Rhizome')
moma_rhz_compare_gender_rep = group_genders(moma_rhz_compare, 'MoMA')

fig_1 = px.pie(rhz_gender_rep, values='Counts', names='Gender',
             title='Gender Breakdown of Rhizome Collection')
fig_1.update_traces(textposition='auto', textinfo='label+percent', hoverinfo='label+value+percent', hovertemplate=None, pull=[0, 0, 0, 0.2])
fig_1.update_layout(width=800)

fig_2 = px.pie(moma_rhz_compare_gender_rep, values='Counts', names='Gender',
             title='Gender Breakdown of MoMA Collection (Sampled)')
fig_2.update_traces(textposition='auto', textinfo='label+percent', hoverinfo='label+value+percent', hovertemplate=None, pull=[0, 0, 0, 0.2])
fig_2.update_layout(width=800)

fig_1.show(), fig_2.show()

(None, None)

**SUNBURST CHARTS FOR COLLECTIVE GENDER BREAKDOWN**

In [81]:
rhizome_gender_counts = rhz_artworks['Gender'].value_counts()
#choose only collectives
rhizome_gender_collectives = rhizome_gender_counts[3:].reset_index()
#group collectives categories
rhizome_gender_collectives_male = rhizome_gender_collectives[(rhizome_gender_collectives['index'].str.contains('F') == False) & (rhizome_gender_collectives['index'].str.contains('missing') == False)]
rhizome_gender_collectives_female =rhizome_gender_collectives[(rhizome_gender_collectives['index'].str.contains('M') == False) & (rhizome_gender_collectives['index'].str.contains('missing') == False)]
rhizome_gender_collectives_both = rhizome_gender_collectives[(rhizome_gender_collectives['index'].str.contains('F')== True) & (rhizome_gender_collectives['index'].str.contains('M')== True) & (rhizome_gender_collectives['index'].str.contains('missing')== False)]
rhizome_gender_collectives_male_missing = rhizome_gender_collectives[(rhizome_gender_collectives['index'].str.contains('M') == True) & (rhizome_gender_collectives['index'].str.contains('missing') == True) & (rhizome_gender_collectives['index'].str.contains('F') == False)]
rhizome_gender_collectives_female_missing = rhizome_gender_collectives[(rhizome_gender_collectives['index'].str.contains('M') == False) & (rhizome_gender_collectives['index'].str.contains('missing') == True) & (rhizome_gender_collectives['index'].str.contains('F') == True)]
rhizome_gender_collectives_both_missing = rhizome_gender_collectives[(rhizome_gender_collectives['index'].str.contains('F')== True) | (rhizome_gender_collectives['index'].str.contains('M')== True) | (rhizome_gender_collectives['index'].str.contains('missing')== True)]
#send filters to series 
rhizome_gender_collectives_both_counts = rhizome_gender_collectives_both.sum()
rhizome_gender_collectives_both_missing_counts = rhizome_gender_collectives_both_missing.sum()
rhizome_gender_collectives_male_counts = rhizome_gender_collectives_male.sum()
rhizome_gender_collectives_female_counts = rhizome_gender_collectives_female.sum()
rhizome_gender_collectives_male_missing_counts = rhizome_gender_collectives_male_missing.sum()
rhizome_gender_collectives_female_missing_counts = rhizome_gender_collectives_female_missing.sum()
#create new DF w/ results 
rhizome_gender_collectives_counts_final = pd.DataFrame()
rhizome_gender_collectives_counts_final['Male Only'] = rhizome_gender_collectives_male_counts
rhizome_gender_collectives_counts_final['Male & Missing'] = rhizome_gender_collectives_male_missing_counts
rhizome_gender_collectives_counts_final['Female Only'] = rhizome_gender_collectives_female_counts
rhizome_gender_collectives_counts_final['Female & Missing'] = rhizome_gender_collectives_female_missing_counts
rhizome_gender_collectives_counts_final['Mixed Gender'] = rhizome_gender_collectives_both_counts
rhizome_gender_collectives_counts_final['Mixed Gender incl. Missing'] = rhizome_gender_collectives_female_missing_counts
rhizome_gender_collectives_counts_final = rhizome_gender_collectives_counts_final.drop('index', axis=0)

In [85]:
moma_gender_counts = moma_rhz_compare['Gender'].value_counts()
#choose only collectives
moma_gender_collectives = moma_gender_counts[3:].reset_index()
#group collectives categories
moma_gender_collectives_male = moma_gender_collectives[(moma_gender_collectives['index'].str.contains('F') == False) & (moma_gender_collectives['index'].str.contains('missing') == False) & (moma_gender_collectives['index'].str.contains('NB') == False)]
moma_gender_collectives_female =moma_gender_collectives[(moma_gender_collectives['index'].str.contains('M') == False) & (moma_gender_collectives['index'].str.contains('missing') == False) & (moma_gender_collectives['index'].str.contains('NB') == False)]
moma_gender_collectives_both = moma_gender_collectives[(moma_gender_collectives['index'].str.contains('F')== True) & (moma_gender_collectives['index'].str.contains('M')== True) & (moma_gender_collectives['index'].str.contains('missing')== False)]
moma_gender_collectives_male_missing = moma_gender_collectives[(moma_gender_collectives['index'].str.contains('M') == True) & (moma_gender_collectives['index'].str.contains('missing') == True) & (moma_gender_collectives['index'].str.contains('F') == False)]
moma_gender_collectives_female_missing = moma_gender_collectives[(moma_gender_collectives['index'].str.contains('M') == False) & (moma_gender_collectives['index'].str.contains('missing') == True) & (moma_gender_collectives['index'].str.contains('F') == True)]
moma_gender_collectives_both_missing = rhizome_gender_collectives[(rhizome_gender_collectives['index'].str.contains('F')== True) | (rhizome_gender_collectives['index'].str.contains('M')== True) | (rhizome_gender_collectives['index'].str.contains('missing')== True)]
#send filters to series 
moma_gender_collectives_both_counts = moma_gender_collectives_both.sum()
moma_gender_collectives_male_counts = moma_gender_collectives_male.sum()
moma_gender_collectives_female_counts = moma_gender_collectives_female.sum()
moma_gender_collectives_male_missing_counts = moma_gender_collectives_male_missing.sum()
moma_gender_collectives_female_missing_counts = moma_gender_collectives_female_missing.sum()
moma_gender_collectives_both_missing_counts = moma_gender_collectives_both_missing.sum()
#create new DF w/ results 
moma_gender_collectives_counts_final = pd.DataFrame()
moma_gender_collectives_counts_final['Male Only'] = moma_gender_collectives_male_counts
moma_gender_collectives_counts_final['Male & Missing'] = moma_gender_collectives_male_missing_counts
moma_gender_collectives_counts_final['Female Only'] = moma_gender_collectives_female_counts
moma_gender_collectives_counts_final['Female & Missing'] = moma_gender_collectives_female_missing_counts
moma_gender_collectives_counts_final['Mixed Gender'] = moma_gender_collectives_both_counts
moma_gender_collectives_counts_final['Mixed Gender incl. Missing'] = moma_gender_collectives_both_missing_counts
moma_gender_collectives_counts_final = moma_gender_collectives_counts_final.drop('index', axis=0)

In [94]:
#collective insight RHIZOME
import plotly.graph_objects as go

fig_sun =go.Figure(go.Sunburst(
    labels=["Collectives", "Males", "Male Only", "Male & Missing", "Females", "Female Only", "Female & Missing", "Mixed Gender", 'Including Missing Gender'],
    parents=["", "Collectives","Males", "Males", "Collectives", "Females", "Females", "Collectives", 'Mixed Gender'],
    values=[133, 39, 37, 2, 11, 8, 3, 83, 3]
),layout=dict(title=dict(text="Gender Breakdown of Rhizome Collectives")))
fig_sun.update_traces(textinfo='label+value', hoverinfo='label+value', hovertemplate=None)
fig_sun.update_layout(width=1000, height=1200)

fig_sun_2 =go.Figure(go.Sunburst(
    labels=["Collectives", "Males", "Male Only", "Male & Missing", "Females", "Female Only", "Female & Missing", "Mixed Gender", 'Including Missing Gender'],
    parents=["", "Collectives","Males", "Males", "Collectives", "Females", "Females", "Collectives", 'Mixed Gender'],
    values=[1634, 903, 435, 468, 206, 135, 71, 525, 131]
),layout=dict(title=dict(text="Gender Breakdown of MoMA Collectives")))
fig_sun_2.update_traces(textinfo='label+value', hoverinfo='label+value', hovertemplate=None)
fig_sun_2.update_layout(width=1000, height=1200)

fig_sun.show(), fig_sun_2.show()

(None, None)

**GENDER BREAKDOWN PIE CHARTS MoMA B4 VS MoMA AFTER**

In [105]:
#create a DF w/ genders and counts, group collectives together
moma_b4_gender_rep = group_genders(moma_artworks_old, 'Rhizome')
moma_after_gender_rep = group_genders(moma_artworks_new, 'MoMA')

fig_1 = px.pie(moma_b4_gender_rep, values='Counts', names='Gender',
             title='Gender Breakdown of MoMA Collection Before 1983')
fig_1.update_traces(textposition='inside', textinfo='label+percent', hoverinfo='label+value+percent', hovertemplate=None, pull=[0, 0, 0, 0.2])
fig_1.update_layout(width=800)

fig_2 = px.pie(moma_after_gender_rep, values='Counts', names='Gender',
             title='Gender Breakdown of MoMA Collection After 1983')
fig_2.update_traces(textposition='inside', textinfo='label+percent', hoverinfo='label+value+percent', hovertemplate=None, pull=[0, 0, 0, 0.2])
fig_2.update_layout(width=800)

fig_1.show(), fig_2.show()

(None, None)

**GENDER REP MoMA BY DEPT BAR CHARTS**

In [106]:
moma_draw_before_gender_rep = group_genders(moma_draw_before, 'MoMA')
moma_draw_after_gender_rep = group_genders(moma_draw_after, 'MoMA')
moma_films_before_gender_rep = group_genders(moma_films_before, 'MoMA')
moma_films_after_gender_rep = group_genders(moma_films_after, 'MoMA')
moma_media_before_gender_rep = group_genders(moma_media_before, 'MoMA')
moma_media_after_gender_rep = group_genders(moma_media_after, 'MoMA')
moma_photo_before_gender_rep = group_genders(moma_photo_before, 'MoMA')
moma_photo_after_gender_rep = group_genders(moma_photo_after, 'MoMA')

In [107]:
moma_draw_before_gender_rep['Period'] = 'Before 1980'
moma_draw_after_gender_rep['Period'] = 'After 1980'
moma_draw_before_gender_rep['Department'] = 'Drawing & Print'
moma_draw_after_gender_rep['Department'] = 'Drawing & Print'
moma_draw_before_gender_rep['Counts'] = moma_draw_before_gender_rep['Counts'].div(100).round(2)
moma_draw_before_gender_rep['Counts'] = moma_draw_before_gender_rep['Counts'].multiply(30).round(2)
moma_draw_before_gender_rep['Counts'] = moma_draw_before_gender_rep['Counts'].astype(int)
moma_draw_gender_rep_all = pd.concat([moma_draw_before_gender_rep, moma_draw_after_gender_rep])
moma_films_before_gender_rep['Period'] = 'Before 1980'
moma_films_after_gender_rep['Period'] = 'After 1980'
moma_films_before_gender_rep['Department'] = 'Films'
moma_films_after_gender_rep['Department'] = 'Films' 
moma_films_before_gender_rep['Counts'] = moma_films_before_gender_rep['Counts'].div(100).round(2)
moma_films_before_gender_rep['Counts'] = moma_films_before_gender_rep['Counts'].multiply(30).round(2)
moma_films_before_gender_rep['Counts'] = moma_films_before_gender_rep['Counts'].astype(int)
moma_films_gender_rep_all = pd.concat([moma_films_before_gender_rep, moma_films_after_gender_rep])
moma_media_before_gender_rep['Period'] = 'Before 1980'
moma_media_after_gender_rep['Period'] = 'After 1980'
moma_media_before_gender_rep['Department'] = 'Media & Performance'
moma_media_after_gender_rep['Department'] = 'Media & Performance' 
moma_media_before_gender_rep['Counts'] = moma_media_before_gender_rep['Counts'].div(100).round(2)
moma_media_before_gender_rep['Counts'] = moma_media_before_gender_rep['Counts'].multiply(30).round(2)
moma_media_before_gender_rep['Counts'] = moma_media_before_gender_rep['Counts'].astype(int)
moma_media_gender_rep_all = pd.concat([moma_media_before_gender_rep, moma_media_after_gender_rep])
moma_photo_before_gender_rep['Period'] = 'Before 1980'
moma_photo_after_gender_rep['Period'] = 'After 1980'
moma_photo_before_gender_rep['Department'] = 'Photography'
moma_photo_after_gender_rep['Department'] = 'Photography' 
moma_photo_before_gender_rep['Counts'] = moma_photo_before_gender_rep['Counts'].div(100).round(2)
moma_photo_before_gender_rep['Counts'] = moma_photo_before_gender_rep['Counts'].multiply(30).round(2)
moma_photo_before_gender_rep['Counts'] = moma_photo_before_gender_rep['Counts'].astype(int)
moma_photo_gender_rep_all = pd.concat([moma_photo_before_gender_rep, moma_photo_after_gender_rep])
moma_depts_gender_rep_all = pd.concat([moma_photo_gender_rep_all, moma_media_gender_rep_all, moma_films_gender_rep_all, moma_draw_gender_rep_all])
moma_depts_gender_rep_all = moma_depts_gender_rep_all.reset_index(drop='True')

In [122]:
fig = px.bar(moma_depts_gender_rep_all, x="Gender", y="Counts", color="Period", facet_col="Department", facet_col_spacing=0.06,  facet_col_wrap=2, title="Gender Rep in MoMA Departments before and after 1980", barmode='group', log_y=True, text_auto=True)
fig.update_xaxes(matches=None, showticklabels=True)
fig.update_yaxes(showticklabels=True)
fig.update_traces(textposition="outside")
fig.update_layout(height=800, width=1600)
fig.show()

**GENDER REP RHZ VS MOMA SAMPLE BY DEPT**

In [162]:
#create a list of rhizome mediums based on keywords research 
rhz_medium = rhizome_txt_stop_kw.copy()
# create a list of our conditionsP
conditions = [
    (rhz_medium['Text'].str.contains('video|film|animation|movie|tv', regex=True)),
    (rhz_medium['Text'].str.contains('images|image|photographs|photography|photo|photos|painting|drawing', regex=True)),
    (rhz_medium['Text'].str.contains('installation|performance|sound|audio|song|music', regex=True)),
    (rhz_medium['Text'].str.contains('website|flash|java|data|database|webpage|net|internet|site|sites|online|webbased|augmented|virtual|reality|game|interaction|interactive|interface|project|xml|mashup|map|generative|wifi|network|email|computer', regex=True)),
    ]

# create a list of the values we want to assign for each condition
values = ['Moving Images', 'Still Images', 'Installation & Performance', 'Net Art']

# create a new column and use np.select to assign values to it using our lists as arguments
rhz_medium['Medium'] = np.select(conditions, values)

# display updated DataFrame
rhz_medium['Medium'] = rhz_medium['Medium'].replace('0', 'missing')
rhz_medium['Medium'] = rhz_medium['Medium'].astype(str)


In [175]:
rhz_gender_rep_mi = group_genders(rhz_medium.loc[rhz_medium['Medium'] == 'Moving Images'], 'Rhizome')
rhz_gender_rep_img = group_genders(rhz_medium.loc[rhz_medium['Medium'] == 'Still Images'], 'Rhizome')
rhz_gender_rep_intall = group_genders(rhz_medium.loc[rhz_medium['Medium'] == 'Installation & Performance'], 'Rhizome')
rhz_gender_rep_netart = group_genders(rhz_medium.loc[rhz_medium['Medium'] == 'Net Art'], 'Rhizome')
rhz_gender_rep_mi['Department'] = 'Moving Images'
rhz_gender_rep_img['Department'] = 'Still Images'
rhz_gender_rep_intall['Department'] = 'Installation'
rhz_gender_rep_netart['Department'] = 'Net Art'
rhz_rep_all = pd.concat([rhz_gender_rep_mi, rhz_gender_rep_img, rhz_gender_rep_intall, rhz_gender_rep_netart])
rhz_rep_all['Source'] = 'Rhizome'

In [ ]:
moma_depts_gender_rep_all_2 = moma_depts_gender_rep_all.loc[moma_depts_gender_rep_all['Period'] == 'After 1980']
moma_depts_gender_rep_all_2['Counts'] = moma_depts_gender_rep_all_2['Counts'].div(10).round(2)
moma_depts_gender_rep_all_2['Counts'] = moma_depts_gender_rep_all_2['Counts'].astype(int)
moma_depts_gender_rep_all_2['Source'] = 'MoMA'
moma_rhz_gender_compare = pd.concat([rhz_rep_all, moma_depts_gender_rep_all_2])
moma_rhz_gender_compare = moma_rhz_gender_compare.drop('Period', axis=1)

In [177]:
fig = px.bar(moma_rhz_gender_compare, x="Gender", y="Counts", color="Department", facet_col="Source", facet_col_spacing=0.06,  facet_col_wrap=2, title="Gender Rep Between Rhizome and MoMA Departments", barmode='group', log_y=True, text_auto=True)
fig.update_xaxes(matches=None, showticklabels=True)
fig.update_yaxes(showticklabels=True)
fig.update_traces(textposition="outside")
fig.update_layout(height=800, width=1600)
fig.show()